In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xscen

from clisops.core.subset import subset_shape

from xhydro.pmf.pmf import remove_precip, separate_pr, fix_pmp_year, place_pmf_inputs, maximize_snow

In [ ]:
from IPython.display import clear_output

import xhydro as xh
import xhydro.modelling as xhm

clear_output(wait=False)

In [ ]:
# Workaround for determining the notebook folder within a running notebook
# This cell is not visible when the documentation is built.

from __future__ import annotations

try:
    from _finder import _find_current_folder

    notebook_folder = _find_current_folder()
except ImportError:
    from pathlib import Path

    notebook_folder = Path().cwd()

import logging

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


* On charge la météo
* On définit la pmp
* On choisit une année de Base
* On place la pmp
* On enlève la précip solide avant
* On enlève la précip solide après
* On instancie un model
* On fait une sim
* On trouve le facteur de neige pour Neige 100
* On applique le facteur 
* On simule
*
*
*
*

# PMF Analysis with Different Tools

In this notebook, we will perform some things that can be done in a Probable Maximum Flood (PMF) analysis. A PMF analysis is not a straigthfoward méthodology, this notebook is thus intended to be an example of different methods. For each methods, we will try to give a brief explanations and highlights its purpose.

---





### 3. **Choose a base year**
   A reference year is selected to align the meteorological data. This ensures that the PMP event is placed in a consistent context, allowing for accurate comparisons and simulations.

   **Purpose**: To provide a baseline for integrating the PMP into the weather data.

---

### 4. **Place the PMP**
   The PMP event is integrated into the weather data at the appropriate time. This step modifies the precipitation data to include the extreme event, ensuring that the PMP is properly represented in the analysis.

   **Purpose**: To simulate the impact of the PMP on the hydrological system.

---

### 5. **Remove solid precipitation before**
   Any solid precipitation (e.g., snow) that occurred before the PMP event is removed. This step isolates the impact of the PMP by eliminating prior snow accumulation.

   **Purpose**: To focus the analysis on the PMP event and its direct effects.

---

### 6. **Remove solid precipitation after**
   Similarly, solid precipitation that occurred after the PMP event is removed. This ensures that the analysis remains centered on the PMP and avoids interference from subsequent snow events.

   **Purpose**: To maintain the integrity of the PMP-focused analysis.

---

### 7. **Instantiate a model**
   A hydrological model is created and configured. This model will simulate the behavior of the hydrological system under the influence of the PMP and other factors.

   **Purpose**: To provide a computational framework for simulating the PMF.

---

### 8. **Run a simulation**
   An initial simulation is performed using the prepared data. This provides a baseline scenario to evaluate the system's response to the PMP.

   **Purpose**: To establish a reference point for further analysis and adjustments.

---

### 9. **Find the snow factor for Snow 100**
   The snow factor for a specific scenario (e.g., Snow 100) is calculated. This involves determining the contribution of snow to the overall hydrological response.

   **Purpose**: To quantify the role of snow in the PMF analysis and adjust the model accordingly.

---

### 10. **Apply the snow factor**
   The calculated snow factor is applied to the data. This step modifies the input data to account for the effects of snow on the hydrological system.

   **Purpose**: To incorporate the snow factor into the analysis and improve the accuracy of the simulation.

---

### 11. **Run the simulation**
   A new simulation is performed with the adjusted data. This step evaluates the hydrological system's response to the PMP, including the effects of the snow factor.

   **Purpose**: To generate the final results of the PMF analysis and assess the system's behavior under extreme conditions.

---

By following these steps, we systematically analyze the Probable Maximum Flood scenario, leveraging various tools and methods to ensure a comprehensive and accurate assessment.

### 1. **Define the PMP**
   Let define some inputs for the PMF, here we'll define :
   * Probable Maximum Precipitation (pmp); 
   * 100 year return period precipitation (p100); 
   * critical temperature sequences (min_temp, mean_temp, max_temp);
   * probable maximum snow accumulation (pmsa);
   * 100 year return period snow (swe100). 

In [ ]:


# Define input dictionaries
pmsa = {
    'Catchment-1': 443,
}
pmp = {
    'C1': {
        'Catchment-1': {'J1': 32.0, 'J2': 191.1, 'J3': 38.8},
    },

}
t_crit_17_31_mai = {
    'max_temp': {
        'Catchment-1': [16.7, 17.3, 17.6, 17.8, 18.0, 18.1, 18.4, 18.8, 26.3, 29.5, 33.3, 20.5, 4.5, 5.0, 4.0],
    },
    'mean_temp': {
        'Catchment-1': [10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 10.9, 11.2, 15.3, 16.9, 18.9, 12.2, 4.5, 5.0, 4.0],
    },
    'min_temp': {
        'Catchment-1': [4.9, 4.3, 4.0, 3.8, 3.6, 3.5, 3.4, 3.6, 4.3, 4.3, 4.5, 3.9, 4.5, 5.0, 4.0],
    }
}

# Prepare arrays and coordinates
bvs = list(pmsa.keys())
time = pd.date_range(start='2010-05-17', periods=15, freq='D')
center = list(pmp.keys())

data = {

    'pmsa': (['bv'], [pmsa[st] for st in bvs]),
    'max_temp': (['bv', 'time'], np.zeros((len(bvs), 15))),
    'mean_temp': (['bv', 'time'], np.zeros((len(bvs), 15))),
    'min_temp': (['bv', 'time'], np.zeros((len(bvs), 15))),
    'pmp': (['bv', 'time', 'center'], np.zeros((len(bvs), 15, len(center)))),
}

for i, st in enumerate(bvs):
    data['max_temp'][1][i, :] = t_crit_17_31_mai['max_temp'][st]
    data['mean_temp'][1][i, :] = t_crit_17_31_mai['mean_temp'][st]
    data['min_temp'][1][i, :] = t_crit_17_31_mai['min_temp'][st]
    for j, center_key in enumerate(center):
        data['pmp'][1][i, -3:, j] = [
            pmp[center_key][st]['J1'],
            pmp[center_key][st]['J2'],
            pmp[center_key][st]['J3']
        ]

ds_pmf_inputs = xr.Dataset(data, coords={'bv': bvs, 'time': time, 'center': center})
ds_pmf_inputs


### 2. **Load the weather data**
   In this step, we import the meteorological data required for the analysis. This includes precipitation, temperature, and other relevant variables. The data is typically loaded from a file or a database and is structured to cover the time period of interest. We here choose daily ERA5_land


In [ ]:
path_to_file = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
era = xr.open_dataset(path_to_file)
era = era[["tas", "tasmin","tasmax","pr"]]
era

Lets clip our meteorology on the Kipawa Watershed and lets only keep 10 years of data

In [ ]:
gdf = gpd.read_file('Kipawa_poly.json')
gdf = gdf.to_crs("epsg:4326")
date_debut = '2010-01-01'
date_fin = '2020-01-01'
ds_meteo = subset_shape(era, shape=gdf, start_date=date_debut, end_date=date_fin).mean(dim=['lat', 'lon'])
ds_meteo

After the clipping, our dataset lost its attribute, lets put them back as we'll need them for unit conversion

In [ ]:
for var in ds_meteo:
    ds_meteo[var].attrs = era[var].attrs
ds_meteo

We put everything in °C and mm 

In [ ]:
ds_meteo = xscen.utils.change_units(ds_meteo, {"tas": "degC", "tasmin": "degC", "tasmax": "degC","pr": "mm"})
ds_meteo

Our `ds_pmf_inputs` was arbitrarily placed at 2010, if we want to apply the pmp to another year, the function `fix_pmp_year` does just that. For example here 2012. It can be applied on the dataset or on a data array

In [ ]:
ds_pmp = fix_pmp_year(ds_pmf_inputs, 2012)
ds_pmp

In [ ]:
da_pmp = fix_pmp_year(ds_pmf_inputs['pmp'], 2012)
da_pmp

To place inputs in the loaded meteorology, both datasets need to have matching names, here we place the pmp, but it could be a p100 or whatever we want to place.

In [ ]:
pmp = ds_pmf_inputs[['max_temp', 'mean_temp', 'min_temp', 'pmp']].rename({'max_temp':'tasmax', 'min_temp':'tasmin', 'mean_temp':'tas', 'pmp':'pr'})
pmp

Now that both datasets have matching variables and units, we can place the inputs in our meteo dataset. 

In [ ]:
ds = place_pmf_inputs(pmp, ds_meteo)
ds

We can see that the pmp was indeed place in may 2012

In [ ]:
ds.pr.plot()

For the temperature, it's not so obvious, but if we recall, we had this sequence : <br>
**10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 10.9, 11.2, 15.3, 16.9, 18.9, 12.2, 4.5, 5.0, 4.0** for mean temperature from may 17 to mai 31. <br>
Plotting a zoomed period, and showing the sequence between the red lines, we see that it's been replaced correctly as well

In [ ]:


# Plot the data
ax = ds.tas.sel(time=slice('2012-05-10', '2012-06-05')).plot()
may_17 = pd.Timestamp('2012-05-17')
may_31 = pd.Timestamp('2012-05-31')
# Add red dotted vertical lines at May 17 and May 31
plt.axvline(x=may_17, color='red', linestyle='--')
plt.axvline(x=may_31, color='red', linestyle='--')


# Show the plot
plt.show()

Another thing that is often needed is to remove precipitation, it can be used to add a dry period before or after our event <br>
The `remove_precip` function does that, it takes the desired dataarray and a `pandas.date_range`. It would also work on temperature (or any other variable), but I'm not sure when that could be relevant. By default it will only remove for the dates specified, but it can remove specific date everyyear

In [ ]:
time = pd.date_range(start='2012-06-01', periods=150, freq='D')
remove_precip(ds['pr'], time, each_year=False).plot()

In [ ]:
remove_precip(ds['pr'], time, each_year=True).plot()

In [ ]:
hru = xh.gis.watershed_to_raven_hru(watershed=gdf, unique_id='ID_POULPE')

Depending on the models, you may need to feed it total precipitation or snow and rainfall. While total precipitation can be obtanined from rainfall and snowfall with a straigtfooward sum, spliiting precipitation into snow and rain is a little more complex. We implemented two formulas to do the splitting <br> The input variable is kept but two new variable appended _rain and _snow are added. The algorithm used and the temperature transition specified is also kept in the metadata. 

In [ ]:
ds_rs = separate_pr(ds,'pr','tasmin','tasmax').drop('pr')
ds_rs

In [ ]:
hru

In [ ]:
ds_rs = ds_rs.assign_coords({"altitude": 450, "lat": 46, "lon": -72})
ds_rs["altitude"].attrs["units"] = "m"
ds_rs["lat"].attrs["units"] = "degrees_north"
ds_rs["lon"].attrs["units"] = "degrees_east"
ds_rs

In [ ]:
ds_rs.squeeze().dims

In [ ]:
# You can also use the 'save_as' argument to save the new file(s) in your project folder.
ds_reformatted, config = xh.modelling.format_input(
    ds_rs.rename({'pr_snow':'snowfall', 'pr_rain':'rainfall'}),
    "GR4JCN",
    save_as=notebook_folder / "_data" / "meteo_hmr.nc",
)
ds_reformatted

In [ ]:
model_config = {
    "model_name": "GR4JCN",
    "parameters": [
        0.529,
        -3.396,
        407.29,
        1.072,
        16.9,
        0.947,
    ],  # GR4JCN has 6 parameters, others might have more
    "global_parameter": {"AVG_ANNUAL_SNOW": 100.00},
    "hru": hru,
    "start_date": "2010-01-01",
    "end_date": "2013-12-31",
    "RainSnowFraction": "RAINSNOW_DINGMAN",
    "Evaporation": "PET_HARGREAVES_1985",
    **config,
}

In [ ]:
ht = xhm.hydrological_model(model_config)
ht

In [ ]:
ds_out = ht.run()
ds_out

In [ ]:
maximize_snow(ds_reformatted, model_config,2012, 422)